In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoModel
from datasets import load_dataset
from tqdm import tqdm
import random
from huggingface_hub import login
import pandas as pd
import numpy as np
import json
import gc
import os

In [2]:
with open('D:\Programming\My_reps\Hugging_face_token_my.txt', 'r') as f:
    hf_token = f.readlines()[0]
login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\Otniel\.cache\huggingface\token
Login successful


In [3]:
# quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=200.0)
if 1:
    model_id = "google/gemma-2-9b-it"
    Gemma = True
else:
    model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    Gemma = False
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="cuda").eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
if Gemma:
    model.generation_config.pad_token_id = tokenizer.pad_token_id
    model.generation_config.pad_token_id = model.generation_config.eos_token_id
    model.generation_config.cache_implementation = "static"
    Terminators = [model.generation_config.eos_token_id, 107]

else:
    # tokenizer.pad_token_id = 128004  # "<|finetune_right_pad_id|>"
    Terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>"), tokenizer.pad_token_id, 128009, 128004]

In [5]:
def ask_model(prompt, max_new_tokens=500, cut_input=True, temperature=0.2, max_batch_tokens=1600):
    params = {"max_new_tokens": max_new_tokens, "use_cache": True, "eos_token_id":Terminators}
    if temperature > 0:
        params.update({'temperature': temperature, 'do_sample': True, 'top_p': 0.9})
    else:
        params.update({'do_sample': False})
        
    if isinstance(prompt, str):
        batch = False
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        batch_parts = [input_ids]
        
    elif isinstance(prompt, list) or isinstance(prompt, np.ndarray):
        batch = True
        prompt = [p for p in prompt]
        input_ids = tokenizer.batch_encode_plus(prompt, padding=False, return_attention_mask=False)
        input_ids = input_ids['input_ids']

        lengths = [len(ii) for ii in input_ids]
        lengths_ind = np.argsort(lengths)
        lengths_ind_rev = np.argsort(lengths_ind)

        input_ids = [input_ids[ii] for ii in lengths_ind]
        lengths = np.sort(lengths)

        batch_parts = []
        cur_lens = [lengths[0]]
        id_new = [0]
        for i in range(1, len(lengths)):
            new_max = max(cur_lens + [lengths[i]]) * (len(cur_lens) + 1)
            if new_max > max_batch_tokens:
                batch_parts.append(tokenizer.pad({'input_ids': input_ids[min(id_new): max(id_new) + 1]}, return_tensors='pt', 
                                                 return_attention_mask=False)['input_ids'])
                id_new = [i]
                cur_lens = [lengths[i]]
            else:
                id_new.append(i)
                cur_lens.append(lengths[i])
        batch_parts.append(tokenizer.pad({'input_ids': input_ids[min(id_new): max(id_new) + 1]}, return_tensors='pt', 
                                         return_attention_mask=False)['input_ids'])
    
    else:
        raise BaseException(f"Wrong prompt format: {type(prompt)}")

    if cut_input:
        cut_input = [len(p) for p in prompt] if batch else len(prompt)
    else:
        cut_input = [0] * len(prompt) if batch else 0

    with torch.inference_mode():
        outputs_all = []
        for b_part in batch_parts:
            output = model.generate(b_part.to('cuda') , **params).to('cpu') # '<end_of_turn>'
            outputs_all.append(output)
            torch.cuda.empty_cache()
            gc.collect()

    del(input_ids)
    
    if batch:
        outputs = []
        for out in outputs_all:
            outputs.extend(tokenizer.batch_decode(out, skip_special_tokens=True))
        return [outputs[b][cut_input[i]:].strip() for i, b in enumerate(lengths_ind_rev)]
    else:
        outputs = tokenizer.decode(outputs_all[0][0], skip_special_tokens=True)
        return outputs[cut_input:].strip()

In [6]:
out = ask_model('I`m the social analyst. Here 10 private optimistic thoughts for human 20-40 ages with reasoning and details from the first person in one monologue:\n',
                temperature=0.4, max_new_tokens=500)
print(out)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"You know, sometimes when I'm scrolling through the endless feed of doom and gloom, I have to remind myself of the good stuff.  Like, sure, the world's got its problems, but look at us! We're the generation that grew up with the internet, with instant access to information and connection. We've seen incredible technological advancements, from smartphones to renewable energy. We're more aware of social issues than ever before, and we're actively fighting for change.

I see it every day. People are passionate about making the world a better place. They're volunteering, donating, and speaking out against injustice.  We're not just consumers; we're creators, innovators, and activists. We're building communities, supporting each other, and challenging the status quo.

And let's not forget about the sheer joy of being alive! We have the freedom to explore, to learn, to love, and to create. We can travel the world, experience different cultures, and connect with people from all walks of life.

---

# Load texts and summarize

In [7]:
def summ_prompt_lesson(text):
    if isinstance(text, list):
        return [summ_prompt_lesson(t) for t in text]
    prompt = "**Ты еврейский раввин пишешь краткое пересказ на часть урока. У тебя в основном текст на русском языке, \
но иногда встречается цитаты на иврите и иногда цитаты на ивритском языке написаны русскими буквами, попробуй догадаться о чем речь.\nТебе нужно написать абзац пересказ части урока на русском языке, \
упоминая важные детали. Не нужно приветствий и вступлений, ты делаешь сухой пересказ на среднюю часть урока.**\n\n**текст**\n"
    return prompt + text + "\n\n**Твой пересказ**\n"

In [8]:
# whole Lesson summarization
folder = r"D:\Programming\My_experiments\Audio\texts_from_audio"
for filename in tqdm(os.listdir(folder)):
    file_path = os.path.join(folder, filename)
    ds = pd.read_csv(file_path)
    text = ds.loc[:, 'text'].tolist()
    prompt = summ_prompt_lesson(text)
    summarization = ask_model(prompt=prompt, max_new_tokens=500, temperature=0, cut_input=True)
    ds.loc[:, 'summarization'] = summarization
    ds.to_csv(file_path, index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [1:04:13<00:00, 77.07s/it]


In [9]:
print(summarization)

['Сегодня мы изучаем раздел Вав в Семане Самахбаб из Шуханаруха.  В нем рассматривается ситуация, когда человек, читающий молитву, прерывается и произносит "Эмет".  В этом случае возникает сомнение, нужно ли ему повторить "Эмет" или нет.  Автор Мишны Брура приводит примеры из Талмуда и других источников, чтобы прояснить этот вопрос.  Он указывает, что если человек читает вместе с Хазаном, то нет необходимости повторять "Эмет".  Но если он прервался, то возникает сомнение.  Автор приводит несколько правил, которые помогут решить эту проблему.', 'В тексте обсуждается важность понимания смысла при чтении Криат Шма. Раввин сравнивает чтение Криат Шма с чтением срочного указа царя, который вызывает трепет и ожидание. Он приводит пример Рамаврама, который писал о чтении Криат Шма как о чтении особого постановления.  Важно, чтобы читающий Криат Шма представлял себе, что он читает указ Верховного Совета, и ощущал соответствующее напряжение и трепет.', 'В тексте обсуждается важность медленного 

In [14]:
ds

,start,end,text,summarization
0,0.00,177.38,Мишна Брура Мы решили сегодня проверить по Алл...,Сегодня мы изучаем раздел Вав в Семане Самахба...
1,177.38,359.98,самого начала и говорит так и крак рио тьма Хр...,В тексте обсуждается важность понимания смысла...
2,359.98,538.14,Все время правительственный указ какой-то. Вес...,В тексте обсуждается важность медленного и вни...
3,538.14,713.32,"И дальше здесь мы читаем далее. Гимель, но гим...","В тексте обсуждается порядок чтения молитвы ""Ш..."
4,713.32,897.50,"Хойль бешумин ми пив, шел шалех цибур шалош тр...","В тексте обсуждается вопрос о количестве слов,..."
5,897.50,1079.44,ויש שכתבו דקולה קורא קריאת שמה ביחיד או יאמר א...,В тексте обсуждается вопрос о необходимости по...
6,1079.44,1260.10,"Видите? Вы смотрите в шаханарух, я говорю заин...",В тексте обсуждается порядок произнесения слов...
7,1260.10,1454.08,"поговоришь про себя, повторяешь эти слова, они...",В ходе урока обсуждалась практика повторения с...
8,1454.08,1613.86,"‫הנציא. Вот здесь возьмем Раверстайн, это в ко...","Раввин обсуждает правила бен апраким, то есть ..."
9,1613.86,1797.20,отвечать на приветствие это мы сейчас изучаем ...,Сегодня мы изучаем тему ответов на приветствие...


In [6]:
ds['summarization'].apply(len) / ds['text'].apply(len)

0     0.366451
1     0.299381
2     0.207847
3     0.295143
4     0.334520
5     0.242865
6     0.294790
7     0.162398
8     0.307853
9     0.285033
10    0.231469
11    0.184673
12    0.274409
13    0.383254
14    0.282652
15    0.284314
16    0.240617
17    0.142917
18    0.323367
19    0.211778
20    0.428571
21    0.333333
22    0.376279
23    0.424671
dtype: float64

In [15]:
model.cpu()
del(tokenizer, model)
torch.cuda.empty_cache()
gc.collect()

0

---